In [1]:
import os
import numpy as np
import pandas as pd

os.chdir("../..")
import utils

In [2]:
outputfolder = os.getcwd() + '/output/'
datafolder = os.getcwd() + '/datasets/PTB-XL/'
sampling_rate = 100
task = 'priority'
experiment_name = 'exp4'

data, raw_labels = utils.load_dataset(datafolder, sampling_rate=sampling_rate)
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
data, labels, Y, _ = utils.select_data(data, labels, task, 0, outputfolder+experiment_name+'/data/')

In [3]:
X_test = data[labels.strat_fold == 10]
y_test = Y[labels.strat_fold == 10]

X_val = data[labels.strat_fold == 9]
y_val = Y[labels.strat_fold == 9]

X_train = data[labels.strat_fold <= 8]
y_train = Y[labels.strat_fold <= 8]

n_classes = y_train.shape[1]

print(f"This experiment has {n_classes} classes")

X_train_lead1 = X_train[:,:,0]
X_test_lead1 = X_test[:,:,0]
X_val_lead1 = X_val[:,:,0]

This experiment has 43 classes


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, LeakyReLU
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  Input, Reshape
from tensorflow.keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D,MaxPool1D,AveragePooling1D
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
import numpy as np
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential

In [5]:
lenet_5_model=Sequential()
lenet_5_model.add(Conv1D(filters=6, kernel_size=3, padding='same', input_shape=(1000,1)))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(Conv1D(filters=16, strides=1, kernel_size=5))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(GlobalAveragePooling1D())
lenet_5_model.add(Dense(64, activation='relu'))
lenet_5_model.add(Dense(32, activation='relu'))
lenet_5_model.add(Dense(n_classes, activation = 'sigmoid'))

In [6]:
lenet_5_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1000, 6)           24        
                                                                 
 batch_normalization (BatchN  (None, 1000, 6)          24        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 1000, 6)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 500, 6)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 496, 16)           496       
                                                                 
 batch_normalization_1 (Batc  (None, 496, 16)          6

In [6]:
lenet_5_model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[tf.keras.metrics.AUC(num_thresholds=1000000, multi_label=False),
                utils.CustomMetric(beta=0.5)]
)

In [7]:
history = lenet_5_model.fit(X_train_lead1, y_train, validation_data=(X_val_lead1, y_val), epochs=30)

Epoch 1/30
534/534 [==============================] - ETA: 0s - loss: 0.1229 - auc: 0.9165 - custom_metric: 0.9428

c:\Users\agarw\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric CustomMetric implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


534/534 [==============================] - 33s 57ms/step - loss: 0.1229 - auc: 0.9165 - custom_metric: 0.9428 - val_loss: 0.1194 - val_auc: 0.9187 - val_custom_metric: 0.9431
Epoch 2/30
534/534 [==============================] - 29s 54ms/step - loss: 0.1069 - auc: 0.9397 - custom_metric: 0.9560 - val_loss: 0.1667 - val_auc: 0.8494 - val_custom_metric: 0.9066
Epoch 3/30
534/534 [==============================] - 28s 52ms/step - loss: 0.1043 - auc: 0.9436 - custom_metric: 0.9581 - val_loss: 0.1420 - val_auc: 0.8895 - val_custom_metric: 0.9278
Epoch 4/30
534/534 [==============================] - 29s 53ms/step - loss: 0.1022 - auc: 0.9468 - custom_metric: 0.9598 - val_loss: 0.1039 - val_auc: 0.9454 - val_custom_metric: 0.9585
Epoch 5/30
534/534 [==============================] - 27s 51ms/step - loss: 0.1011 - auc: 0.9484 - custom_metric: 0.9607 - val_loss: 0.1060 - val_auc: 0.9419 - val_custom_metric: 0.9574
Epoch 6/30
534/534 [==============================] - 29s 54ms/step - loss: 0.099

In [14]:
from sklearn.metrics import roc_auc_score

preds = lenet_5_model.predict(X_test_lead1)

print(roc_auc_score(y_test,preds, average='samples'))
auc = lenet_5_model.evaluate(X_test_lead1, y_test)
print(auc)

68/68 [==============================] - 2s 22ms/step
0.9284279656368928
 2/68 [..............................] - ETA: 7s - loss: 0.1119 - auc: 0.9361 - custom_metric: 0.9529 

c:\Users\agarw\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric CustomMetric implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


68/68 [==============================] - 8s 117ms/step - loss: 0.1185 - auc: 0.9323 - custom_metric: 0.9491
[0.1185152530670166, 0.9323394298553467, 0.9490586519241333]
